In [21]:
import pandas as pd
import numpy as np
import os
import pickle

In [4]:
# define current working directory so we can navigate folders easily
cwd = os.getcwd()
os.chdir('../')
pwd = os.getcwd()

In [29]:
def fold_ecfp_from_KNIME(file, length)->None:
    
    df = pd.read_csv(file)
    
    df2 = df[~df['Core (stucture) ECFP fingerprint'].isna()]
    
    ecfp_list = []
    for i in df2['Core (stucture) ECFP fingerprint']:
        
        unfolded = np.array(i[1:-1].split(',')).astype(np.int64)
        
        f = lambda x: abs(x) % length
        indices = f(unfolded)
        
        bitvector = np.zeros(length, dtype=int)
        
        bitvector[indices]=1
        ecfp_list.append(bitvector)
    
    output_df = pd.DataFrame(ecfp_list)
    df2.reset_index(inplace=True)
    output_df['SMILES'] = df2['Reactants>Reagents>Products']
    output_df['Yield'] = df2['Yield (numerical)']
    output_file = "{}_ECFP_{}.csv".format(file[:-4], length)
    pickle_file = "{}_ECFP_{}.pkl".format(file[:-4], length)
    
    output_df.to_csv(output_file, index=False)
    output_df.to_pickle(pickle_file, protocol=4)

In [30]:
fold_ecfp_from_KNIME('/home/gah/oprd/data/oprd_mapped_cores.csv', 512)

In [18]:
pd.read_pickle('/home/gah/oprd/data/oprd_mapped_cores_ECFP_512.pkl').iloc[:,0:-2].to_numpy()

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
data = pickle.load(open(pwd+'/data/oprd_mapped_cores_ECFP_512.pkl',"rb"))

In [28]:
data.iloc[:,-2]

0       [Br-].C[N+]1=CN(CC2=NC(CBr)=CC=C2)C2=CC=CC=C12...
1       [Br-].C[N+]1=CN(CC2=NC(CBr)=CC=C2)C2=CC=CC=C12...
2       [Cu+].[13C-]#[15N].FC1=C(Cl)C=C(I)C=C1>>FC1=C(...
3       [H][C@@](O)(CO)[C@@]([H])(O)[C@]([H])(O)[C@@](...
4       [K+].CC(C)(C)[O-].ClC(=O)C1=CC=C(Br)C=C1>>CC(C...
                              ...                        
2733    SC1=CC=CC=C1Cl.CC(=C)OC(C)=O>OS(=O)(=O)C(F)(F)...
2734    SC1=CC=CC=C1N=CC1=CC=CC=C1>OO>S1C2=CC=CC=C2N=C...
2735    SC1=NN=CN1C1=CC=C(C2CC2)C2=C1C=CC=C2.OC(=O)CBr...
2736    SC1=NNC=N1.CC1=CC(C)=C(N)C(C)=C1>Cl.[Na+].[O-]...
2737    SC1CCCCC1.CC(=C)OC(C)=O>OS(=O)(=O)C(F)(F)F>CC(...
Name: SMILES, Length: 2738, dtype: object